In [ ]:
"""
Description:
    Project 1: N-Grams Language Model

    Project for Amirkabir University of Technilogy (Tehran Polytechnic), Computer Scince department
    Natural Language Processing course

Student Name & ID: Pouria Alimoradpor 403112088
"""

import os
import re
import time
from collections import defaultdict, Counter
import numpy as np
import pandas as pd

In [2]:
data_path = {
    'Tasnim': './Datasets/Tasnim/tasnim.csv',
    'Shahnameh': './Datasets/Shahnameh/shahname.csv',
    'Hamshahri': './Datasets/HamshahriData/HamshahriCorpus'
}

**Data: Tasnim**

In [3]:
data_tasnim = pd.read_csv(data_path['Tasnim'], encoding='utf-8')
data_tasnim.head()

,category,title,abstract,body,time
0,سیاسی,میرکاظمی: زمان حذف ارز ۴۲۰۰ تومانی مشخص نیست,رئیس سازمان برنامه و بودجه گفت: هر زمان شرایط...,به گزارش گروه پارلمانی ، «مسعود میرکاظمی» رئی...,۲۴ فروردين ۱۴۰۱ - ۰۹:۵۰
1,سیاسی,طرح ۲ فوریتی شفافیت قوای سه‌گانه با ۲۰۰ امضا ...,نماینده نیشابور در مجلس از ارائه طرح ۲ فوریتی...,احسان ارکانی نماینده مردم نیشابور در مجلس شورا...,۲۴ فروردين ۱۴۰۱ - ۰۹:۵۰
2,سیاسی,رئیسی انتخاب نخست وزیر جدید پاکستان را تبریک ...,رئیس جمهور کشورمان طی پیامی انتخاب نخست وزیر ...,به گزارش حوزه دولت ، آیت‌الله سید ابراهیم رئی...,۲۵ فروردين ۱۴۰۱ - ۱۳:۵۴
3,سیاسی,اصلاح اساسنامه شرکت شهر فرودگاهی امام خمینی (...,اساسنامه شرکت شهر فرودگاهی امام خمینی(ره) در ...,به گزارش حوزه دولت ، در جلسه صبح روز چهارشنبه...,۳۱ فروردين ۱۴۰۱ - ۱۲:۰۳
4,سیاسی,ارائه طرحی کلی و مبهم برای شفافیت فرار از مطا...,نماینده مردم تهران گفت: در شرایطی که طرح شفاف...,علی خضریان نماینده تهران در مجلس در گفت‌وگو با...,۳۰ فروردين ۱۴۰۱ - ۱۶:۱۵


In [4]:
data_tasnim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63494 entries, 0 to 63493
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  63494 non-null  object
 1   title     63493 non-null  object
 2   abstract  63493 non-null  object
 3   body      62558 non-null  object
 4   time      63493 non-null  object
dtypes: object(5)
memory usage: 2.4+ MB


In [5]:
data_tasnim = data_tasnim.drop(columns=['category', 'time'])
data_tasnim['text'] = data_tasnim['title'] + '. ' + data_tasnim['abstract'] + ' ' + data_tasnim['body']
data_tasnim = data_tasnim.drop(columns=['title', 'abstract', 'body'])
data_tasnim.head()

,text
0,میرکاظمی: زمان حذف ارز ۴۲۰۰ تومانی مشخص نیست ...
1,طرح ۲ فوریتی شفافیت قوای سه‌گانه با ۲۰۰ امضا ...
2,رئیسی انتخاب نخست وزیر جدید پاکستان را تبریک ...
3,اصلاح اساسنامه شرکت شهر فرودگاهی امام خمینی (...
4,ارائه طرحی کلی و مبهم برای شفافیت فرار از مطا...


In [6]:
print(data_tasnim['text'][0])

 میرکاظمی: زمان حذف ارز ۴۲۰۰ تومانی مشخص نیست .  رئیس سازمان برنامه و بودجه گفت: هر زمان شرایط و فضای اقتصادی کشور مساعد باشد، می‌توان به تدریج حذف ارز ترجیحی را عملیاتی کرد و در حال حاضر برای حذف ارز ۴۲۰۰ تومانی زمان و تاریخ مشخصی را نمی‌توان تعیین کرد.  به گزارش گروه پارلمانی  ، «مسعود میرکاظمی» رئیس سازمان برنامه و بودجه، در پاسخ به سؤالی مبنی بر اینکه دولت از چه زمانی اقدام به حذف ارز 4200 تومانی خواهد کرد، گفت: براساس قانون بودجه سال 1401، اختیار حذف و یا عدم حذف ارز ترجیحی برعهده دولت قرار داده شده است و از سوی دیگر حجم ارز 4200 تومانی در کشور به میزانی نیست که بتوان تا پایان سال از آن استفاده کرد. وی که با خانه ملت گفت‌وگو کرده است ادامه داد: بنابراین هر زمان شرایط و فضای اقتصادی کشور مساعد باشد، می‌توان به تدریج این موضوع را عملیاتی کرد و در حال حاضر برای حذف ارز ترجیحی زمان و تاریخ مشخصی را نمی‌توان تعیین کرد. رئیس سازمان برنامه و بودجه در پاسخ به سؤال دیگری مبنی بر اینکه چرا با توجه به اینکه هنوز ارز ترجیحی حذف نشده است، شاهد افزایش قیمت کالاها به ویژه در مورد گوشت قرمز و مرغ

In [7]:
unique_chars = set()
for text in data_tasnim['text'].dropna():
    unique_chars.update(set(text))
print(unique_chars)

{'ً', '２', '✌', 'Ş', '\u2003', '４', '۸', 'ﮒ', 'ﺾ', 'ύ', 'ۀ', '\u200d', '=', 'J', 'р', 'б', 'I', 'ø', 'ۖ', 'к', 'ﻈ', 'ﮕ', 'É', 'ﺵ', '５', 'С', 'ھ', 'ú', 'د', 'ﮔ', 'ﻄ', '\u202c', 'ﭼ', 'ı', 'ﺶ', 'ٔ', '🔸', 'ﮐ', 'γ', '^', '۵', 'β', 'e', 'í', 'i', 'ﻚ', 'y', '−', 'в', 't', 'ü', 'O', 'ز', 'ۗ', 'ﻣ', 'ﻌ', 'е', 'ﺗ', 'f', 'а', '+', '📱', 'ۙ', 'r', 'υ', '\u2069', '≈', 'َ', '\u200a', 'ﺻ', '﴾', '#', '”', 'κ', 'ق', 'ﮏ', 'ﺘ', 'ڼ', '⁵', 'آ', '\u2028', '۰', 'A', 'ó', '۳', 'p', 'ο', 'ﻢ', '٫', '٩', 'Ε', '👈', 'ٴ', '2', 'ب', 'ﻤ', '|', '\uf0d8', 'Æ', 'n', 'ﺤ', '`', 'ι', 'ة', 'w', 'ﻓ', 'ﻕ', 'z', '\x88', 'c', 'ﺝ', 'ы', 'ڋ', '\u202b', 'g', 'ﻛ', 'ﻭ', 'ّ', 'ﻻ', 'ﮓ', 'ﻍ', 'ﻆ', 'ﻨ', 'é', 'ğ', 'ﻱ', 'N', 'Ρ', 'q', '…', 'ﯾ', '9', 'G', '‼', '‹', 'ﭽ', 'ﭻ', 'ﻐ', 'ﻼ', '\u200b', 'ﻑ', 'Q', 'ت', '６', 'ﻯ', 'ﻘ', 'ٕ', 'ν', 'd', 'õ', 'ﮑ', '🔴', 'ﺕ', '3', 'ﻰ', '[', 'ﺌ', '⦁', '۱', 'ﺨ', 'ﺏ', 'á', 'ﺣ', 'ﻊ', 'Á', 'ە', '🌹', 'α', '؍', 'ٌ', '!', 'ﻒ', '۔', 'ﻳ', 'ﻉ', '/', 'ﻇ', 'ﺢ', 'ﺲ', 'ع', 'ﻟ', 'ﻸ', 'ﷲ', '😒', '☑', 'ﺼ', 'ﺐ', '٢', '\x9d', 'ﻡ'

**Data: Shahnameh**

In [8]:
data_shahnameh = pd.read_csv(data_path['Shahnameh'], encoding='utf-8')
data_shahnameh.head()

,Chapter,Part,Bait,Mesra,Text
0,1,1,1,1,به نام خداوند جان و خرد
1,1,1,1,2,کز این برتر اندیشه بر نگذرد
2,1,1,2,1,خداوند نام و خداوند جای
3,1,1,2,2,خداوند روزی ده رهنمای
4,1,1,3,1,خداوند کیوان و گَردان سپهر


In [9]:
data_shahnameh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99222 entries, 0 to 99221
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Chapter  99222 non-null  int64 
 1   Part     99222 non-null  int64 
 2   Bait     99222 non-null  int64 
 3   Mesra    99222 non-null  int64 
 4   Text     99222 non-null  object
dtypes: int64(4), object(1)
memory usage: 3.8+ MB


In [10]:
data_shahnameh = data_shahnameh.drop(columns=['Chapter', 'Part', 'Bait', 'Mesra'])
data_shahnameh = data_shahnameh.rename(columns={'Text': 'text'})
data_shahnameh.head()

,text
0,به نام خداوند جان و خرد
1,کز این برتر اندیشه بر نگذرد
2,خداوند نام و خداوند جای
3,خداوند روزی ده رهنمای
4,خداوند کیوان و گَردان سپهر


In [11]:
unique_chars = set()
for text in data_shahnameh['text']:
    unique_chars.update(set(text))
print(unique_chars)

{'ع', 'ج', 'ز', 'ّ', 'م', 'ن', 'ف', 'چ', 'ث', ')', 'ئ', 'ژ', 'ش', 'ذ', 'س', 'ط', 'ظ', ' ', 'ء', '(', 'َ', 'غ', 'ا', 'ُ', 'خ', 'پ', 'ه', 'ل', 'ح', 'ق', 'ی', '؛', 'گ', 'ت', 'ر', 'آ', '\u200c', 'د', '«', 'ٔ', '،', '»', 'ص', 'أ', ':', 'ؤ', 'و', '؟', 'ک', 'ض', 'ب', '\xa0', 'ي', '!', 'ْ', 'ِ'}


**Data: Hamshahri**

In [12]:
data_hamshahri = []
for year_folder in os.listdir(data_path['Hamshahri']):
    folder_path = os.path.join(data_path['Hamshahri'], year_folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith(".ham"):
                file_path = os.path.join(folder_path, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    data_hamshahri.append(f.read())

In [13]:
data_hamshahri = pd.DataFrame(data_hamshahri, columns=['text'])
data_hamshahri.head()

,text
0,جودي ابوت، نابغه كاغذي\nسميه نصيري ها\nجودي اب...
1,برنامه ريزي براي مرگ\nنگاهي به استرس و ساز و ك...
2,آغاز ساخت بدنه و سرريز بلندترين سد\nكشور\nعملي...
3,عوامل موثر در پيشگيري از ديابت\nدكتر محمود بهز...
4,آغاز عمليات ساخت مترو تجريش\nگروه شهري: كلنگ ا...


In [14]:
data_hamshahri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5375 entries, 0 to 5374
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5375 non-null   object
dtypes: object(1)
memory usage: 42.1+ KB


In [15]:
unique_chars = set()
for text in data_hamshahri['text']:
    unique_chars.update(set(text))
print(unique_chars)

{'_', 'ً', 'U', '.', 'م', 'ٍ', 'ف', 'ش', '\u200d', 'ء', 'J', '=', 'غ', 'I', 'ح', 'گ', 'S', 'د', 'b', '\x7f', '~', 'e', '\xa0', 'l', 'i', 'y', 'ك', 'o', 't', 'ج', 'ز', 'f', 'ث', '+', ';', 'r', 'Y', 'َ', 'ا', '\n', 'پ', 'ه', '•', 'ق', 'L', '؛', '¨', 'آ', 'A', 'F', 'p', '½', 'أ', '?', 'ؤ', 'و', '2', '"', 'D', 'ب', '|', ']', '®', '5', 'n', 'ي', 'ة', 'w', 'z', '%', '&', '@', 'c', '¼', '8', 'g', 'ّ', 'N', 'ç', 'ژ', 'q', 'ظ', '…', 'P', '9', 'G', 'k', ',', 'E', 'Q', 'ت', 'd', 'T', '«', '7', '3', '©', '[', 'ص', 'a', 'H', '-', 'إ', '!', 'â', '/', 'ع', 'j', 'h', 'ن', 'چ', '0', 'K', ')', 'ئ', 'C', 'س', 'ذ', 'ط', 'R', ' ', '(', '\u200e', 'ُ', 'X', 'خ', 'ل', 'm', 'u', 'ر', 'W', 'v', '1', "'", 'M', 'x', '،', '4', '»', '–', 'B', 'è', ':', 'V', 'ى', '؟', '>', 'ض', 's', '*', '}', 'ـ', 'ِ', '6'}


In [16]:
# Save the data to a CSV file
data_tasnim.to_csv('./Datasets/_CSV/tasnim_cleaned.csv', index=False, encoding='utf-8')
data_shahnameh.to_csv('./Datasets/_CSV/shahname_cleaned.csv', index=False, encoding='utf-8')
data_hamshahri.to_csv('./Datasets/_CSV/hamshahri_cleaned.csv', index=False, encoding='utf-8')

**Code**

In [13]:
class DataProcessor:
    """
    A class to process the data for the language model
    """
    def __init__(self, data_path):
        """
        Initialize the DataProcessor object
        ----------------------------------------------
        Args:
            data_path (str): The path to the data file
        Returns:
            None
        """
        self.data_path = data_path
        self.data = []
        self.cleaned_data = []
        self.tokenized_data = []
        self.prepared_data = []

        self.read_data()
        self.clean_text()
        self.tokenizer()
        frequencies_dict, n_tokens, n_unique_tokens = self.get_most_freqs()
        print(f"Number of tokens: {n_tokens}")
        print(f"Number of unique tokens: {n_unique_tokens}")
        self.handle_unknown()

    def read_data(self):
        """
        Read the data from the file
        """
        self.data = pd.read_csv(self.data_path, encoding='utf-8')['text'].dropna().tolist()
    
    def clean_text(self):
        cleaned_data = [re.sub(r'[^آ-ی۰-۹.؟\s]', '', line) for line in self.data]   # Remove non-persian characters
        cleaned_data = [re.sub(r'[۰-۹]+', 'N', line) for line in cleaned_data]      # Replace numbers with 'N'
        cleaned_data = [re.sub(r'\s+', ' ', line) for line in cleaned_data]         # Remove extra spaces
        cleaned_data = [line.strip() for line in cleaned_data]                      # Remove leading and trailing spaces
        self.cleaned_data = cleaned_data
    
    def tokenizer(self):
        """
        Tokenize the cleaned data
        """
        self.tokenized_data = [line.split() for line in self.cleaned_data]
    
    def get_most_freqs(self, top_n=10000):
        """
        Get the most frequent tokens in the data
        ----------------------------------------------
        Args:
            top_n (int): The number of most frequent tokens to keep
        Returns:
            frequencies_dict (dict): A dictionary of the most frequent tokens and their counts
            n_tokens (int): The total number of tokens in the data
            n_unique_tokens (int): The number of unique tokens in the data
        """
        all_tokens = [token for sentence in self.tokenized_data for token in sentence]
        frequencies = Counter(all_tokens)
        most_freq = frequencies.most_common(top_n)
        self.frequencies_dict = dict(most_freq)
        with open('frequent.txt', 'w', encoding='utf-8') as f:
            f.writelines([f"{token}\t{count}\n" for token, count in most_freq[:200]])
        self.n_tokens = len(all_tokens)
        self.n_unique_tokens = len(frequencies.keys())
        return self.frequencies_dict, self.n_tokens, self.n_unique_tokens
    
    def handle_unknown(self):
        """
        Replace the less frequent tokens with '<UNK>'
        """
        self.prepared_data = [[word if word in self.frequencies_dict else "<UNK>" for word in sentence] for sentence in self.tokenized_data]


In [ ]:
class NgramLanguageModel:
    """
    A class to build an N-gram language model
    """
    def __init__(self, prepared_data):
        """
        Initialize the NgramLanguageModel object
        ----------------------------------------------
        Args:
            prepared_data (list): The preprocessed data
        Returns:
            None
        """
        self.prepared_data = prepared_data
        self.unigram_dict = Counter()
        self.bigram_dict = defaultdict(Counter)
        self.trigram_dict = defaultdict(Counter)

        self.make_ngram_dicts()
        print(f"Number of unigrams: {len(self.unigram_dict)}")
        print(f"Number of bigrams: {len(self.bigram_dict)}")
        print(f"Number of trigrams: {len(self.trigram_dict)}")
    
    def make_ngram_dicts(self):
        """
        Create dictionaries for unigrams, bigrams, and trigrams
        """
        for sentence in self.prepared_data:
            for i in range(len(sentence)):
                self.unigram_dict[sentence[i]] += 1
                if i < len(sentence) - 1:    # if i > 0:
                    self.bigram_dict[sentence[i-1]][sentence[i]] += 1
                if i < len(sentence) - 2:    # if i > 1:
                    self.trigram_dict[(sentence[i-2], sentence[i-1])][sentence[i]] += 1
    
    def calculate_smoothed_probs(self, n, input_text):
        """
        Calculate the smoothed probabilities of the next word
        ----------------------------------------------
        Args:
            n (int): The n-gram model to use
            input_text (str): The input text
        Returns:
            probs (dict): A dictionary of the probabilities of the next word
        """
        words = input_text.split()
        probs = {}

        for candidate in self.unigram_dict.keys():
            word = candidate
            if n == 2:
                prev_word = words[-1]
                probs[word] = (self.bigram_dict[prev_word][word] + 1 / (self.unigram_dict[prev_word] + len(self.unigram_dict)))
            elif n == 3:
                prev_bigram = (words[-2], words[-1])
                probs[word] = (self.trigram_dict[prev_bigram][word] + 1 / (self.bigram_dict[prev_bigram[1]][word] + len(self.bigram_dict)))

        return probs
    
    def generate_text(self, n, input_text=None, max_length=20):
        """
        Generate text using the n-gram model
        ----------------------------------------------
        Args:
            n (int): The n-gram model to use
            input_text (str): The input text
            max_length (int): The maximum length of the generated text
        Returns:
            text (str): The generated text
        """
        if not input_text:
            input_text = " ".join(np.random.choice(list(self.unigram_dict.keys()), 3))
            
        text = input_text.split()
        
        print(input_text, end=' ')
        for _ in range(max_length):
            time.sleep(0.01)
            probs = self.calculate_smoothed_probs(n, " ".join(text))
            # probs = self.kneser_ney_smoothing(2, " ".join(text))
            if not probs:
                break
            next_word = max(probs, key=probs.get)
            text.append(next_word)
            print(next_word, end=' ')
        
        return " ".join(text)
    
    def evaluate_model(self, prepared_test_data, n):
        """
        Evaluate the model using the test data
        ----------------------------------------------
        Args:
            prepared_test_data (list): The preprocessed test data
            n (int): The n-gram model to use
        Returns:
            log_likelihood (float): The log-likelihood of the model
            perplexity (float): The perplexity of the model
        """
        log_likelihood = 0
        total_tokens = 0
        
        for sentence in prepared_test_data[:300]:
            if len(sentence) < 3:
                continue
            input_text = " ".join(sentence)
            probs = self.calculate_smoothed_probs(n, input_text)
            
            for p in probs.values():
                log_likelihood += np.log(p)
                total_tokens += 1
        
        avg_log_likelihood = log_likelihood / total_tokens if total_tokens > 0 else float('-inf')
        perplexity = np.exp(-avg_log_likelihood)
        return avg_log_likelihood, perplexity
    
    def kneser_ney_smoothing(self, n, input_text, d=0.75):
        """
        Calculate the Kneser-Ney smoothed probabilities of the next word
        ----------------------------------------------
        Args:
            n (int): The n-gram model to use
            input_text (str): The input text
            d (float): The discount factor
        Returns:
            probs (list): A list of the probabilities of the next word
        """
        words = input_text.split()
        probs = {}

        for candidate in self.unigram_dict.keys():
            if candidate in words:
                continue
            word = candidate
            context = " ".join(words[-n+1:])
            context_words = context.split()
            context_word_counts = {}
            for i in range(1, n):
                context_word_counts[context_words[-i]] = len(self.bigram_dict[context_words[-i]])
            if n == 2:
                probs[word] = max(self.bigram_dict[context_words[-1]][word] - d, 0) / self.unigram_dict[context_words[-1]] + d * len(self.bigram_dict[context_words[-1]]) / self.unigram_dict[context_words[-1]] * self.bigram_dict[context_words[-1]][word]
            else:
                probs[word] = 0
        
        return probs


In [224]:
# tasnim_processor = DataProcessor('./Datasets/_CSV/tasnim_cleaned.csv')
shahname_processor = DataProcessor('./Datasets/_CSV/shahname_cleaned.csv')
# hamshahri_processor = DataProcessor('./Datasets/_CSV/hamshahri_cleaned.csv')

Number of tokens: 567159
Number of unique tokens: 18944


In [229]:
prepapred_data = shahname_processor.prepared_data
train_size = int(0.95 * len(prepapred_data))
train_data = prepapred_data[:train_size]
test_data = prepapred_data[train_size:]

In [266]:
lm = NgramLanguageModel(train_data)

Number of unigrams: 9984
Number of bigrams: 9700
Number of trigrams: 143636


In [267]:
input_text = "به یاد"
print("Generated text:")
# generated_text = lm.generate_text(3, input_text=None, max_length=20)
generated_text = lm.generate_text(3, input_text, max_length=5)
print()
for _ in range(3):
    generated_text = lm.generate_text(3, input_text=(generated_text.split()[-2] + ' ' + generated_text.split()[-1]), max_length=5)
    print()
    

Generated text:
به یاد ندارد جهان چون شب تیره 
شب تیره شد آن نامور گذر کرد 
گذر کرد بر خویشتن تار به خداوند 
به خداوند به خداوند به خداوند به 


In [ ]:
print("Evaluating the model (n=3):")
log_likelihood, perplexity = lm.evaluate_model(test_data, 3)
print(f"Log likelihood: {log_likelihood}")
print(f"Perplexity: {perplexity}")
print()
print("Evaluating the model (n=2):")
log_likelihood, perplexity = lm.evaluate_model(test_data, 2)
print(f"Log likelihood: {log_likelihood}")
print(f"Perplexity: {perplexity}")

Evaluating the model (n=3):
Log likelihood: -9.170771065368235
Perplexity: 9612.03333939397
Evaluating the model (n=2):
Log likelihood: -9.06635798044812
Perplexity: 8659.029951535555


In [253]:
# Kneser-Ney Smoothing
print("Kneser-Ney Smoothing:")
input_text = "چو رستم بیاید به"
probs = lm.kneser_ney_smoothing(2, input_text)

# Print top 5 probabilities (sorted by value)
sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
for word, prob in sorted_probs[:5]:
    print(f"{word}: {prob}")

# Smoothing
print("Add-1 Smoothing:")
probs = lm.calculate_smoothed_probs(3, input_text)

# Print top 5 probabilities (sorted by value)
sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
for word, prob in sorted_probs[:5]:
    print(f"{word}: {prob}")

Kneser-Ney Smoothing:
نزدیک: 27.301329740495575
پیش: 23.0166071114756
هر: 18.237493409876397
کردار: 15.271146974401029
سر: 13.89783843945873
Add-1 Smoothing:
چاره: 1.0001029442042413
از: 1.0001026377912348
درگاه: 1.0001020199959192
جنگ: 1.0001017501017502
ایران: 1.000100969305331
